## Segmenting and Clustering Neighborhoods in Toronto_GR
### Part1
#### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

In [3]:
!pip install bs4
import pandas as pd
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [4]:
#The below url contains html tables with data about postal codes of Canada.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
data  = requests.get(url).text

In [6]:
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table') 

In [7]:
for index,table in enumerate(tables):
    if ("Neighbourhood" in str(table)):
        table_index = index
print(table_index)

0


In [10]:
#print(tables[table_index].prettify())
pc_data = pd.DataFrame(columns=["Postal Code", "Borough", "Neighbourhood"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        postalcode = col[0].text.strip()
        borough = col[1].text.strip()
        neighbourhood = col[2].text.strip()
        pc_data = pc_data.append({"Postal Code":postalcode, "Borough":borough, "Neighbourhood":neighbourhood}, ignore_index=True)

pc_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [14]:
pc_data = pc_data[pc_data["Borough"]!="Not assigned"]

In [15]:
pc_data.shape

(103, 3)

### Part2
#### Get the latitude and the longitude coordinates of each neighborhood

In [16]:
#download the data and open the download data 
read = requests.get("http://cocl.us/Geospatial_data", allow_redirects=True)
open('data.csv', 'wb').write(read.content)
geo_data = pd.read_csv('data.csv')
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
pc_data = pc_data.merge(geo_data, on="Postal Code")
pc_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Part3
#### Generate maps to visualize your neighborhoods and how they cluster together

In [18]:
import numpy as np
from sklearn.cluster import KMeans
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [19]:
pc_data.groupby("Borough").count()

,Postal Code,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


In [21]:
#dropping the group of Borough and postal code
X = pc_data.drop(['Borough','Postal Code'], axis =1)
#X = X.set_index('Neighbourhood')

In [22]:
X.head()

,Neighbourhood,Latitude,Longitude
0,Parkwoods,43.753259,-79.329656
1,Victoria Village,43.725882,-79.315572
2,"Regent Park, Harbourfront",43.654260,-79.360636
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [23]:
# set K cluster to group data into 10 K group based on Borough size
k = 10
df_clustering = X.drop('Neighbourhood',axis=1)
kmeans = KMeans(n_clusters=k, random_state = 1).fit(df_clustering)

#check labels of Kmeans generated
kmeans.labels_[:15]

array([2, 8, 1, 4, 1, 6, 5, 7, 8, 1, 4, 6, 5, 8, 8], dtype=int32)

In [24]:
X.insert(3,'Cluster Labels', kmeans.labels_)
X.head()

,Neighbourhood,Latitude,Longitude,Cluster Labels
0,Parkwoods,43.753259,-79.329656,2
1,Victoria Village,43.725882,-79.315572,8
2,"Regent Park, Harbourfront",43.654260,-79.360636,1
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1


In [25]:
map_clusters = folium.Map(location=[X['Latitude'][0], X['Longitude'][0]])

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(X['Latitude'], X['Longitude'], X['Neighbourhood'], X['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= 10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters